In [1]:
#Libraries:
import pandas as pd
import numpy as np
import os
import time
import librosa
import librosa.display
import matplotlib.pyplot as plt
from pydub import AudioSegment
from scipy.ndimage import zoom
from keras.applications import VGG16
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import cv2 as cv
from keras import layers, models, Model
from keras.optimizers import Adam
import tensorflow as tf
from pydub import AudioSegment
import pickle

In [2]:
!brew install ffmpeg

To reinstall 7.1_2, run:
  brew reinstall ffmpeg


In [29]:
#------------------Function to create spectrogram from audio file------------------
def create_spectrogram(wav_path, sr=22050):

    y, sr = librosa.load(wav_path)
    # Create mel spectrogram
    mel_spect = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_mels=128,  # Number of mel bands
        fmax=8000)    # Maximum frequency
    # Convert to log scale and return
    return np.array(librosa.power_to_db(mel_spect, ref=np.max))


In [30]:
# Function to loop through all clip files and generate spectrograms
def get_features_model (folder_path):
    spectrograms = [] # This will store the spectrograms of each clip
    labels = []  # This will store the labels of each clip
    seconds = []  # Number of seconds to consider for each clip
    durations = []  # Duration of the full audio file
    podcast_names = []  # This will store the podcast names of each clip

    # Iterate over all files in the directory
    file_list = os.listdir(folder_path)
    print(f"Processing files: total {len(file_list)}")
    for filename in file_list:
        # Check if the file is a .wav or .mp3 (you can adjust this as needed)
        if filename.endswith('.wav') or filename.endswith('.mp3'):
            file_path = os.path.join(folder_path, filename)

            # Split the filename by underscore
            filename_parts = filename.split('_')

            # Extract 0 or 1 from the first part of the filename (label: ad or no_ad)
            is_ad = int(filename_parts[0])  # First part is the label

            # Extract the start time in seconds (second part of the filename)
            start_time = int(filename_parts[1])  # Second part is the start time in seconds

            # Extract the total duration (third part of the filename)
            duration = int(filename_parts[2])  # Third part is the total duration of the podcast

             # Extract the podcast name (four part of the filename)
            podcast_name = filename_parts[3].replace('.wav', '')  # Third part is the total duration of the podcast

            # Create spectrogram and convert to numpy array
            spectrogram = create_spectrogram(file_path)

        if spectrogram.shape == (128, 216):

            # Append the numpy array to the list
            spectrograms.append(spectrogram)
            labels.append(is_ad)
            seconds.append(start_time)
            durations.append(duration)
            podcast_names.append(podcast_name)

        else:
            print(f'{filename_parts} is not correct shape')

            # Append the numpy array to the list
            spectrograms.append(spectrogram)
            labels.append(is_ad)
            seconds.append(start_time)
            durations.append(duration)
            podcast_names.append(podcast_name)

    return spectrograms, labels, seconds, durations, podcast_names

In [ ]:
# # DOES NO WORK !!!!!!!!
# # List of folder paths
# folder_paths = [
#     '../raw_data/5_sec_clips/trevornoah_copy',
#     '../raw_data/5_sec_clips/ramitsethi_copy',
#     '../raw_data/5_sec_clips/glucosegoddess_copy',
#     # '../raw_data/5_sec_clips/gabriellelyon',
#     # '../raw_data/5_sec_clips/eyedoctor',
#     # '../raw_data/5_sec_clips/borisjohnson',
# ]

# # Initialize a list to store all spectrograms
# all_spectrograms = []

# # Loop through each folder path
# for folder_path in folder_paths:
#     # Call the get_features_model function for each folder
#     spectrograms = get_features_model(folder_path)

#     # Append the results to all_spectrograms
#     all_spectrograms.extend(spectrograms)

Processing files: total 325
Processing files: total 305
Processing files: total 325


In [32]:
clip_1 = get_features_model('../raw_data/5_sec_clips/trevornoah_copy')
clip_2 = get_features_model('../raw_data/5_sec_clips/ramitsethi_copy')
clip_3 = get_features_model('../raw_data/5_sec_clips/glucosegoddess_copy')


Processing files: total 325
Processing files: total 305
Processing files: total 325


In [33]:
all_spectrograms_clips = clip_1[0] + clip_2[0] + clip_3[0]
all_spectrograms_clips_np = np.array(all_spectrograms_clips)
all_spectrograms_clips_np.shape

(955, 128, 216)

In [8]:
all_labels_clips = clip_1[1] + clip_2[1] + clip_3[1]
all_labels_clips_np = np.array(all_labels_clips)
all_labels_clips_np.shape

(955,)

In [9]:
# Define X and y:
X = all_spectrograms_clips_np
y = all_labels_clips_np

# Print the shape of X and y
print(X.shape)
print(y.shape)

(955, 128, 216)
(955,)


In [10]:
X = np.expand_dims(X, axis=-1)
X.shape

(955, 128, 216, 1)

In [11]:
# Split the data into training and testing sets: 80% training, 20% testing:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,  # This ensures similar class distribution in train/test splits
)

In [12]:
model = tf.keras.models.Sequential([
    # Input layer - note that Input() doesn't take an activation
    tf.keras.layers.Input(shape=(128, 216, 1)),

    # Conv2D layer
    tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'),

    # Flatten layer
    tf.keras.layers.Flatten(),

    # Dense layers
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train model for 10 epochs, capture the history
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping,])

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - accuracy: 0.5384 - loss: 262.7704 - val_accuracy: 0.4974 - val_loss: 22.2376
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.6435 - loss: 12.0079 - val_accuracy: 0.7644 - val_loss: 0.5999
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - accuracy: 0.6902 - loss: 1.7249 - val_accuracy: 0.7696 - val_loss: 0.4945
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.7707 - loss: 0.5271 - val_accuracy: 0.7853 - val_loss: 0.4412
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - accuracy: 0.7894 - loss: 0.4021 - val_accuracy: 0.8063 - val_loss: 0.4149
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.8630 - loss: 0.3317 - val_accuracy: 0.8115 - val_loss: 0.3916
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.8532 - loss: 0.3117 - val_accuracy: 0.8325 - val_loss: 0.3797
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.8629 - loss: 0.2682 - val_accuracy

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 108, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 221184)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    56,623,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 169,871,813 (648.01 MB)

 Trainable params: 56,623,937 (216.00 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 113,247,876 (432.01 MB)

In [14]:
# Evaluate the model:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8467 - loss: 0.3327
Test accuracy: 0.8638743162155151


In [15]:
X_predict = np.expand_dims(create_spectrogram('../raw_data/5_sec_clips/glucosegoddess/0_1_6030_glucosegoddess.wav'), axis=-1)
X_predict_2 = np.expand_dims(create_spectrogram('../raw_data/5_sec_clips/trevornoah/0_1_9615_trevornoah.wav'), axis=-1)
X_predict_3 = np.expand_dims(create_spectrogram('../raw_data/5_sec_clips/glucosegoddess/1_4636_6030_glucosegoddess.wav'), axis=-1)
print(X_predict.shape, X_predict_2.shape, X_predict_3.shape)
processed_input = X_predict.reshape(1, 128, 216, 1)
processed_input_2 = X_predict_2.reshape(1, 128, 216, 1)
processed_input_3 = X_predict_3.reshape(1, 128, 216, 1)
print(processed_input.shape, processed_input_2.shape, processed_input_3.shape)

(128, 216, 1) (128, 216, 1) (128, 216, 1)
(1, 128, 216, 1) (1, 128, 216, 1) (1, 128, 216, 1)


In [16]:
predition = model.predict(processed_input)
predition_2 = model.predict(processed_input_2)
predition_3 = model.predict(processed_input_3)
predition, predition_2, predition_3

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


(array([[0.03654276]], dtype=float32),
 array([[0.3032453]], dtype=float32),
 array([[0.9734764]], dtype=float32))

In [42]:
def create_spectrogram_1(wav_path, sr=22050):
    y, sr = librosa.load(wav_path, sr=sr)
    # Create mel spectrogram
    mel_spect = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_mels=128,  # Number of mel bands
        fmax=8000)    # Maximum frequency
    # Convert to log scale and return
    return np.array(librosa.power_to_db(mel_spect, ref=np.max))

In [54]:
# Function to detect ads within a podcast
def detect_ads(podcast_file_2, model, clip_duration=5):
    """
    This function splits the podcast into clips, creates spectrograms, and uses the model to detect ads.
    podcast_file: Path to the podcast audio file (mp3)
    model: The trained model for ad detection
    clip_duration: Duration of each clip in seconds (default 5)
    return: List of ad segments (start_time, end_time) in seconds
    """
    podcast = AudioSegment.from_file(podcast_file_2)  # Load podcast file
    podcast_duration = len(podcast) / 1000  # Duration in seconds
    print(f"Podcast duration: {podcast_duration} seconds.")

    ad_segments = []  # Store detected ad segments

    # Process the podcast in chunks
    for i in range(0, int(podcast_duration), clip_duration):
        start_time = i * 1000  # Convert to milliseconds
        end_time = (i + clip_duration) * 1000
        clip = podcast[start_time:end_time]

        # Save the clip as a temporary wav file for processing
        clip_file = "temp_clip.wav"
        clip.export(clip_file, format="wav")

        # Create and validate spectrogram
        spectrogram = create_spectrogram_1(clip_file)
        if spectrogram.shape == (128, 216):
            spectrogram = np.expand_dims(spectrogram, axis=-1)  # Add channel dimension
            spectrogram = np.expand_dims(spectrogram, axis=0)   # Add batch dimension
        else:
            print(f"Clip starting at {start_time / 1000} seconds has incorrect shape: {spectrogram.shape}")
            os.remove(clip_file)
            continue  # Skip this clip if shape is incorrect

        # Predict using the model
        prediction = model.predict(spectrogram)
        print(f"Prediction for clip starting at {start_time / 1000} seconds: {prediction}")

        # If the model predicts an ad, add to ad_segments
        if prediction[0][0] > 0.5:  # Assuming a threshold of 0.5
            ad_segments.append((i, i + clip_duration))

        # Remove the temporary file
        os.remove(clip_file)

    print(f"Total ad segments detected: {len(ad_segments)}")
    return ad_segments

In [55]:
# Example of using the functions:
podcast_file = "../raw_data/new_podcast_ceo/Ramit Sethi - Never Split The Bill, It's A Red Flag & Renting Isn't Wasting Money.mp3" # Path to the new podcast file
podcast_file_2 = "../raw_data/new_podcast_ceo/Glucose Goddess -  The Scary New Research On Sugar!.mp3"
ad_segments = detect_ads(podcast_file_2, model)  # Use trained model here

Podcast duration: 6030.811 seconds.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Prediction for clip starting at 0.0 seconds: [[0.03886246]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction for clip starting at 5.0 seconds: [[0.05467738]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Prediction for clip starting at 10.0 seconds: [[0.05054577]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediction for clip starting at 15.0 seconds: [[0.06795312]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediction for clip starting at 20.0 seconds: [[0.08113877]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediction for clip starting at 25.0 seconds: [[0.04137351]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Prediction for clip starting at 30.0 seconds: [[0.03253466]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Prediction for clip starting at 35.0 seconds: [[0.0633781]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction for clip starting at 40.0 seconds: [[0.03955699]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction for clip starting at 45.0 s

In [56]:
def remove_ads_from_podcast(podcast_file, ad_segments):
    """
    Removes the ad segments from the podcast and saves two files:
    - An ad-free podcast file
    - A file with only the ad segments
    podcast_file: Path to the podcast audio file
    ad_segments: List of tuples with (start_time, end_time) of ads in seconds
    """

    # Load
    podcast = AudioSegment.from_file(podcast_file) # Load the podcast file
    podcast_duration = len(podcast)

    # Initialize segments for the ad-free podcast and for the ads only
    clean_podcast = AudioSegment.empty()
    ads_only = AudioSegment.empty()
    current_time = 0

    for ad_start, ad_end in ad_segments:
        ad_start_ms = ad_start * 1000 # Convert to milliseconds
        ad_end_ms = ad_end * 1000
        # Add non-ad segment to the clean podcast
        clean_podcast += podcast[current_time:ad_start_ms]

        # Add ad segment to the ads_only file
        ads_only += podcast[ad_start_ms:ad_end_ms]

        # Update the current time
        current_time = ad_end_ms

    # Add the last segment of the podcast
    clean_podcast += podcast[current_time:podcast_duration]  # Add the last segment of the podcast

    return clean_podcast, ads_only

In [58]:
clean_podcast, ads_only = remove_ads_from_podcast(podcast_file_2, ad_segments)

 # Export the ad-free podcast
clean_podcast.export('podcast_without_ads_2.mp3', format='mp3')
print("Saved ad-free podcast as podcast_without_ads.mp3")

# Export the ads-only segments
ads_only.export('ads_only_2.mp3', format='mp3')
print("Saved ads-only segments as ads_only.mp3")

Saved ad-free podcast as podcast_without_ads.mp3
Saved ads-only segments as ads_only.mp3


In [63]:
model.save('ad_detection_model.h5')